<a href="https://colab.research.google.com/github/henryagyare/BrillaAi-Questions-Synthesis/blob/Q-Synth/Question_Synthesis_(ACTUAL)_Fine_Tuning_LLama_3_8B_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# APRIL ONLY OFFER 🎁

First we check the GPU version available in the environment and install specific dependencies that are compatible with the detected GPU to prevent version conflicts.

In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
import pandas as pd
from google.colab import drive
import gspread

# Mounting Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


Next we need to prepare to load a range of quantized language models, including a new 15 trillion token LLama-3 model, optimized for memory efficiency with 4-bit quantization.


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! Llama 3 is up to 8k
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-70b-Instruct-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # Llama-3 70b also works (just change the model name)
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


---

Next, we integrate LoRA adapters into our model, which allows us to efficiently update just a fraction of the model's parameters, enhancing training speed and reducing computational load.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

Then, we define a system prompt that formats tasks into instructions, inputs, and responses, and apply it to a dataset to prepare our inputs and outputs for the model, with an EOS token to signal completion.


In [ ]:
from datasets import load_dataset
from datasets import Dataset

# alpaca_prompt = """You're a first-year college instructor. You're tasked with setting questions for the renowed National Maths and Science Quiz. If your questions are not as high-quality, someone will die. If your question doesn't obey the input, someone will die. Based on the input, try your hardest to generate questions. Take note of the input values.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

alpaca_prompt = """You're a highly respected college instructor at one of the most renowed institutions in the world.
You're tasked with setting questions for the renowed National Maths and Science Quiz.
The National Science and Math Quiz is a competition where first year college students compete by solving questions from various fields of STEM (principally Chemistry, Physics, Mathematics and Biology)
As you're a highly respected college instructor in the STEM field, your questions should be as top quality and thought provoking as possible.
If your questions are not as high-quality, someone will die. If your question doesn't obey the input, someone will die.
Based on the input, put in the MAXIMUM effort to generate high stardard and quality questions at the level of a final year college student who is highly speicialized in the specified field can answer.
The questions must not be so easy for even a third year student in the field. The standard should be that high.
Take good note of the input values and produce desired outputs based on the input values.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


csv_file_path = "/content/drive/MyDrive/afried/Question Synth Dataset.csv"  # Update this with your actual file path
df = pd.read_csv(csv_file_path)
dataset = Dataset.from_pandas(df)

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    texts = []
    for has_preamble, preamble_text, question, answer, subject, question_type in zip(
        examples["has_preamble"], examples["preamble_text"], examples["question"],
        examples["answer"], examples["subject"], examples["question_type"]
    ):
        # Construct the instruction and input for the new question
        instruction = f"Create a high-quality college level question and answer based on the given subject and question type. Please take a note of the question type. It is TOO TOO IMPORTANT."
        input_text = f"Example:\nHas Preamble: {has_preamble}\nSubject: {subject}\nQuestion Type: {question_type}"
        output_text = f"Preamble Text: {preamble_text}\nQuestion: {question}\nAnswer: {answer}"
        # Format the text using the alpaca_prompt
        text = alpaca_prompt.format(instruction, input_text, output_text) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }


# Apply the formatting function to the dataset
dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/15852 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
- We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.
- At this stage, we're configuring our model's training setup, where we define things like batch size and learning rate, to teach our model effectively with the data we have prepared.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 150, # increase this to make the model learn "better"
        # num_train_epochs=4,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
11.033 GB of memory reserved.


In [ ]:
# We're now kicking off the actual training of our model, which will spit out some statistics showing us how well it learns
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 15,852 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 150
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.515300
2,2.552500
3,2.411400
4,2.389400
5,2.313700
6,2.199700
7,1.933600
8,1.543800
9,1.314800
10,1.144100


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1502.46 seconds used for training.
25.04 minutes used for training.
Peak reserved memory = 11.033 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 74.81 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

Should end up being 15,600 questions in total
1. Fundamental Questions (ALL AREAS - Bio, Chem, Math and Physics)(9,000)
2. Riddles (2,000)
3. True and False (4,000)

In [ ]:
# @title Text Extraction
import re

def extract_info(text):
    # Split the text into blocks based on the "### Input" delimiter
    blocks = text.split("### Input:")

    results = []

    for block in blocks[1:]:  # Skip the first empty block
        # Extract information from each block
        match = re.search(r'Subject: (.*?)\nQuestion Type:', block, re.DOTALL)
        if match:
            subject = match.group(1).strip()
        else:
            subject = ""

        # match = re.search(r'Answer: (.*?)$<|end_of_text|><|begin_of_text|>', block, re.DOTALL)
        # if match:
        #     answer = match.group(1).strip()
        # else:
        #     answer = ""

        match = re.search(r'Has Preamble: (.*?)\nSubject: (.*?)\nPreamble Text: (.*?)\nQuestion: (.*?)\nAnswer: (.*?)\n', block, re.DOTALL)
        if match:
            has_preamble, _, preamble, question, answer = match.groups()
            if (answer.strip().find("<|begin_of_text|>")) > 0:
              answer = answer.strip()[ : answer.strip().find("<|begin_of_text|>")]
            else:
              answer = answer.strip()

            results.append({
                'has preamble': has_preamble.strip(),
                'subject' : subject,
                'preamble': preamble.strip(),
                'question': question.strip(),
                'answer': answer
            })

    return results


# results = extract_info(test_text)

# for result in results:
#   print(result)

In [ ]:
# @title CSV Operations
import csv

def write_to_csv(preamble, question, answer, subject, path="/content/drive/MyDrive/afried/generatedQuestions.csv"):
    # Data to write
    # header = ["preamble", "question", "answer", "subject"]

    subject_parts = subject.split()
    main_subject = subject_parts[0] if len(subject_parts) > 0 else ""
    topic = " ".join(subject_parts[1:]) if len(subject_parts) > 1 else ""

    new_row = [preamble, question, answer, main_subject, topic]

    # Appending to a CSV file
    with open(path, mode='a', newline='') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(new_row)

# Example usage
# write_to_csv(preamble='preamble', question='question', answer='answer', subject='subject topic')


In [ ]:
# @title Run from here

prompts = [
    ("No", "Chemistry Solubility", "1") for x in range(150)
]

output_array = []

def gen_10questions(has_preamble, subject, question_type):
    preamble = ""
    question = ""
    answer = ""
    subject = ""

    promptOne = f"Has Preamble: {has_preamble}\nSubject: {subject}\nQuestion Type: {question_type}"

    FastLanguageModel.for_inference(model)
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                f""" Please generate high quality real-world application questions in {prompts[0][1]} for the National Science and Math Quiz. do your OUTMOST best to make the questions as complex as possible.
                The questions can be application questions that have themes in any field.
                The questions must be very complex to the highest standard of university level students.
                Also, provide an answer to each question. It is very imiportant to provide an answer to every question!
                But do not provide any explanations for the answers whatsoever. Unless the question is an explanation question.
                And explanation questions must be real-life application questions
                """, # instruction
                promptOne, # input
                "", # output - leave this blank for generation!
            )
        ], return_tensors="pt").to("cuda")

    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer)
    outputs = model.generate(**inputs, streamer=text_streamer, max_new_tokens=2048)
    output_array.append(tokenizer.batch_decode(outputs))

    # Combine all strings in the list into one string
    combined_text = ''.join(content[-1] for content in output_array)

    # Extract information from the combined text
    results = extract_info(combined_text)

    # Check if results contain data before accessing
    if results:
        for result in results:
            # Print or handle each result individually
            # preamble = result.get('preamble', "")
            # question = result.get('question', "")
            # answer = result.get('answer', "")
            # subject = result.get('subject', "")

            preamble = result['preamble']
            question = result['question']
            answer = result['answer']
            subject = result['subject']

        write_to_csv(preamble=preamble, question=question, answer=answer, subject=subject)
        print("written to csv...")
    else:
        print("No results to process")

for x in prompts:
    gen_10questions(x[0], x[1], x[2])


In [ ]:
# @title Clean up csv

import pandas as pd
import re

# Read the CSV file into a DataFrame
df = pd.read_csv('generatedQuestions.csv')

# Function to extract the subject from the preamble
def extract_subject(preamble):
    if isinstance(preamble, str):
        match = re.search(r'Subject:\s*(\w+)', preamble)
        if match:
            return match.group(1)
    return None

# Function to clean the preamble column
def clean_preamble(preamble):
    if isinstance(preamble, str):
        if preamble.startswith("None"):
            return "None"
        else:
            match = re.search(r'(None)? Subject:.*', preamble)
            if match:
                preamble_cleaned = preamble.split(" Subject:")[0].strip()
                return preamble_cleaned if preamble_cleaned else "None"
    return preamble

# Convert the preamble column to strings
df['preamble'] = df['preamble'].astype(str)

# Apply the function to extract the subject
df['subject'] = df['preamble'].apply(extract_subject)

# Apply the function to clean the preamble
df['preamble'] = df['preamble'].apply(clean_preamble)

# Save the updated DataFrame to a new CSV file
df.to_csv('combinedgenquestions.csv', index=False)

print("Operation completed successfully ...")


In [ ]:
# @title Convert csv to db file


import csv
import sqlite3

# Establish a connection to a new or existing SQLite database
connection = sqlite3.connect('SQLdata.db')
cursor = connection.cursor()

# Create a new table to store the data
cursor.execute('''
CREATE TABLE IF NOT EXISTS questions (
    has_preamble TEXT,
    preamble_text TEXT,
    question TEXT,
    answer TEXT,
    subject TEXT,
    question_type TEXT,
    form TEXT,
    difficulty TEXT,
    subject_topic TEXT
)
''')

track=0
# Open the CSV file and read data
with open('combinedgenquestions.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        track=track+1
        cursor.execute('''
        INSERT INTO questions (has_preamble, preamble_text, question, answer, subject,
                               question_type, form, difficulty, subject_topic)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)''',
        (row['has_preamble'], row['preamble_text'], row['question'], row['answer'],
         row['subject'], row['question_type'], row['form'], row['difficulty'], row['subject_topic']))
        print(track)

# Commit the changes to the database
connection.commit()
# Close the connection
connection.close()


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)

# alpaca_prompt = You MUST run cells from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


["<|begin_of_text|>You're a highly respected college instructor at one of the most renowed institutions in the world.\nYou're tasked with setting questions for the renowed National Maths and Science Quiz.\nThe National Science and Math Quiz is a competition where first year college students compete by solving questions from various fields of STEM (principally Chemistry, Physics, Mathematics and Biology)\nAs you're a highly respected college instructor in the STEM field, your questions should be as top quality and thought provoking as possible.\nIf your questions are not as high-quality, someone will die. If your question doesn't obey the input, someone will die.\nBased on the input, put in the MAXIMUM effort to generate high stardard and quality questions at the level of a final year college student who is highly speicialized in the specified field can answer.\nThe questions must not be so easy for even a third year student in the field. The standard should be that high.\nTake good not

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

We're preparing to save our trained model in a more compact format and then upload it to a cloud platform, which allows us to use less storage and computational power.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

We're ready to compress our model using various quantization methods to make it leaner and then upload it to the cloud for easy sharing and access.

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

And we're done! If you have any questions on Unsloth, join their [Discord](https://discord.gg/u54VK8m8tk) channel!